In [ ]:
# This is for testing for hysteresis and slop using frames
# with thermal pinhole images

# created 2019 June 23 by E.S.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import gc
from astropy.io import fits
import pandas as pd
import glob
import photutils
from photutils import DAOStarFinder
%matplotlib qt

In [2]:
stem = "/Volumes/seagate_external_drive/lbti_data_reduction/hysteresis_slop/"

In [3]:
def make_median(date_string, start_frame_num, save_name):
    '''
    Make median of five frames, starting with the one with the starting frame number
    '''
    
    # initialize cube
    frame_cube = np.nan*np.ones((5,2048,2048))
    
    for t in range(0,5):
            
        # read in image
        sciImg, header = fits.getdata(stem + "lm_"+date_string+"_"+
                                      '%06i'%(start_frame_num+t)+".fits",
                                      0,header=True)

        # take second slice
        frame_cube[t,:,:] = sciImg[1,:,:]
    
        # take median
        median_frame_cube = np.median(frame_cube, axis = 0).astype(np.uint16)

    hdu = fits.PrimaryHDU(median_frame_cube)
    hdulist = fits.HDUList([hdu])
    hdu.writeto(save_name, overwrite=True)

In [4]:
def make_median_dark_subt(date_string, start_frame_num, save_name):
    '''
    Same as above, but with dark subtraction added in
    '''
    
    # initialize cube
    frame_cube = np.nan*np.ones((5,2048,2048))
    
    for t in range(0,5):
            
        # read in image
        sciImg, header = fits.getdata(stem + "lm_"+date_string+"_"+
                                      '%06i'%(start_frame_num+t)+".fits",
                                      0,header=True)

        # take second slice
        frame_cube[t,:,:] = sciImg[1,:,:]
    
        # take median
        median_frame_cube = np.median(frame_cube, axis = 0).astype(np.uint16)
        
        # dark-subtract
        dark, header = fits.getdata("med_dark.fits",
                                      0,header=True)
        
    frame_dark_subt = np.subtract(median_frame_cube,dark)

    hdu = fits.PrimaryHDU(frame_dark_subt)
    hdulist = fits.HDUList([hdu])
    hdu.writeto(save_name, overwrite=True)

In [7]:
#make_median("190217", 11, "med_dark.fits")
make_median_dark_subt("190217", 16, "med_016_020.fits")
make_median_dark_subt("190217", 21, "med_021_025.fits")
make_median_dark_subt("190217", 26, "med_026_030.fits")
make_median_dark_subt("190217", 31, "med_031_035.fits")
make_median_dark_subt("190217", 36, "med_036_040.fits")
make_median_dark_subt("190217", 41, "med_041_045.fits")
make_median_dark_subt("190217", 46, "med_046_050.fits")

In [8]:
make_median_dark_subt("190218", 6, "med_190218_006_010.fits")
make_median_dark_subt("190218", 11, "med_190218_011_015.fits")
make_median_dark_subt("190218", 16, "med_190218_016_020.fits")
make_median_dark_subt("190218", 21, "med_190218_021_025.fits")
make_median_dark_subt("190218", 26, "med_190218_026_030.fits")
make_median_dark_subt("190218", 31, "med_190218_031_035.fits")
make_median_dark_subt("190218", 36, "med_190218_036_040.fits")
make_median_dark_subt("190218", 41, "med_190218_041_045.fits")
make_median_dark_subt("190218", 46, "med_190218_046_050.fits")
make_median_dark_subt("190218", 51, "med_190218_051_055.fits")
make_median_dark_subt("190218", 56, "med_190218_056_060.fits")
make_median_dark_subt("190218", 61, "med_190218_061_065.fits")
make_median_dark_subt("190218", 66, "med_190218_066_070.fits")
make_median_dark_subt("190218", 71, "med_190218_071_075.fits")
make_median_dark_subt("190218", 76, "med_190218_076_080.fits")
make_median_dark_subt("190218", 81, "med_190218_081_085.fits")
make_median_dark_subt("190218", 86, "med_190218_086_090.fits")
make_median_dark_subt("190218", 91, "med_190218_091_095.fits")
make_median_dark_subt("190218", 96, "med_190218_096_100.fits")
make_median_dark_subt("190218", 101, "med_190218_101_105.fits")
make_median_dark_subt("190218", 106, "med_190218_106_110.fits")
make_median_dark_subt("190218", 111, "med_190218_111_115.fits")
make_median_dark_subt("190218", 116, "med_190218_116_120.fits")
make_median_dark_subt("190218", 121, "med_190218_121_125.fits")

In [54]:
def centroid_psf(file_name, mask=False):
    '''
    Read in an image and get a precise center of the PSF
    '''
    
    plt.clf()
    
    # read in image
    sciImg, header = fits.getdata(file_name,0,header=True)
    
    daofind = DAOStarFinder(fwhm=30, threshold=150, exclude_border=True)
    
    # mask rest of image
    if mask:
        piece_to_keep_top = sciImg[1095:1180,1518:1614]
        piece_to_keep_bottom = sciImg[619:700,1393:1492]
        sciImg = np.median(sciImg)*np.ones(np.shape(sciImg))
        sciImg[1095:1180,1518:1614] = piece_to_keep_top
        sciImg[619:700,1393:1492] = piece_to_keep_bottom   
        
    # kludge for 1 pixel
    sciImg[1135,1567] = sciImg[1135,1566]
    
    sources = daofind(sciImg)
    
    plt.imshow(sciImg, origin="lower")
    plt.scatter([sources["xcentroid"]],[sources["ycentroid"]])
    plt.show()
    
    print(sources)

In [12]:
centroid_psf("med_016_020.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1 1442.61106461 673.060660987 ... 183.0 1.44518064687  -0.399805342709
  2 1557.66587106 1135.60270615 ... 192.0 1.08311592458 -0.0866873528097


In [13]:
centroid_psf("med_021_025.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.43453672  670.14133892 ... 182.0 1.59971679531  -0.51010776114
  2 1562.78755879 1134.99355864 ... 194.0 1.11003485226 -0.113341536843


In [14]:
centroid_psf("med_026_030.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.44280197   669.0143762 ... 186.0  1.6528667337 -0.545594597399
  2 1568.52076201 1134.83130217 ... 190.0 1.10147299002 -0.104934629954


In [15]:
centroid_psf("med_031_035.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1 1442.20903091  667.66081717 ... 188.0 1.70908029866  -0.581906169744
  2 1575.71835279 1134.21268847 ... 191.0  1.0881129479 -0.0916849454491


In [16]:
centroid_psf("med_036_040.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.25644211 667.642581035 ... 183.0 1.72578532962 -0.592466932159
  2 1571.79233546 1133.90616122 ... 190.0 1.09408814917 -0.097630784787


In [17]:
centroid_psf("med_041_045.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.55230489 667.199841294 ... 183.0 1.72898360429 -0.594477187346
  2 1567.45849122 1133.74092645 ... 190.0 1.13239242936 -0.134992393037


In [18]:
centroid_psf("med_046_050.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1 1442.56663814 666.291696191 ... 186.0 1.73544165717  -0.598525044881
  2 1562.59274538 1133.36879173 ... 191.0 1.08995057592 -0.0935170129941


In [19]:
centroid_psf("med_190218_006_010.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1 1442.65971114 665.737573041 ... 181.0 1.77401936806  -0.622395892454
  2 1562.08949482 1138.25411073 ... 190.0 1.07345615313 -0.0769607743706


In [20]:
centroid_psf("med_190218_011_015.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1 1442.62443566 665.298793384 ... 188.0 1.77675446721  -0.624068540104
  2 1561.76388787 1143.69346422 ... 190.0 1.06615581073 -0.0695516955801


In [21]:
centroid_psf("med_190218_016_020.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux            mag        
--- ------------- ------------- ... ----- ------------- ------------------
  1 1442.72449823 664.907066567 ... 186.0  1.7906694888     -0.63253858434
  2 1561.36606657 1150.84746493 ... 192.0 1.00003013983 -3.27234089906e-05


In [25]:
centroid_psf("med_190218_021_025.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag       
--- ------------- ------------- ... ----- ------------- ----------------
  1  1442.5668857 664.600855583 ... 190.0 1.95685258315  -0.728895274753
  2 1561.39267268  1147.6013748 ... 190.0   1.043880546 -0.0466270101194


In [26]:
centroid_psf("med_190218_026_030.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.61019653 664.356085829 ... 189.0 1.96969211826   -0.7359958677
  2 1561.60353536 1142.47336763 ... 191.0  1.1206968452 -0.123720373456


In [27]:
centroid_psf("med_190218_031_035.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1442.49238854 664.010562767 ... 189.0 1.97813247925 -0.740638434381
  2 1561.55022534 1137.83979398 ... 191.0 1.16876784935 -0.169320641127


In [28]:
centroid_psf("med_190218_036_040.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1469.47511727 659.425468574 ... 184.0 1.59812048721 -0.509023797516
  2 1561.06948313 1137.44422162 ... 192.0 1.15575082985 -0.157160534854


In [29]:
centroid_psf("med_190218_041_045.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1449.19735495 659.569348795 ... 186.0 2.10497212409  -0.80811587224
  2 1561.00952415 1137.42924744 ... 192.0 1.17658361696 -0.176556992178


In [30]:
centroid_psf("med_190218_046_050.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1  1449.0130285 654.314716138 ... 188.0 2.04220634672 -0.775249053872
  2 1561.07581152 1137.26372619 ... 191.0 1.16917641496 -0.169700115081


In [31]:
centroid_psf("med_190218_051_055.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1  1448.5298193 658.666255349 ... 187.0 2.13868494531 -0.825367030864
  2 1561.11247591 1137.06071298 ... 194.0 1.15218220388 -0.153802907536


In [32]:
centroid_psf("med_190218_056_060.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1448.43748306 658.474184952 ... 187.0  2.1046024213 -0.807925164271
  2 1560.87766417  1137.1804002 ... 195.0 1.16708727413  -0.16775833389


In [33]:
centroid_psf("med_190218_061_065.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1448.57759423 658.382283421 ... 185.0 2.09717205457 -0.804085154947
  2 1560.79691565 1136.96595144 ... 197.0 1.16264106971 -0.163614150312


In [34]:
centroid_psf("med_190218_066_070.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1448.13624498 658.129367144 ... 185.0 2.11267360442 -0.812081015762
  2 1560.65362422 1136.88188001 ... 192.0 1.16523010161 -0.166029238276


In [35]:
centroid_psf("med_190218_071_075.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1448.02512107 658.089196075 ... 185.0 2.09080928215 -0.800786048704
  2 1560.62359559 1136.98886772 ... 192.0 1.14778339149  -0.14964984046


In [56]:
centroid_psf("med_190218_076_080.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.53503508 657.317302084 ... 184.0 2.68502365666  -1.07237029111
  2 1564.87208182 1136.56457417 ... 192.0 1.64180842794 -0.538306201781


In [57]:
centroid_psf("med_190218_081_085.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.52415548 657.145975123 ... 186.0 2.68631539544  -1.07289250264
  2 1565.48331153 1136.54212592 ... 194.0 1.59976221224 -0.510138585414


In [58]:
centroid_psf("med_190218_086_090.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.42993817 657.045795585 ... 185.0 2.68035730994  -1.07048173077
  2 1565.93015338 1136.67274415 ... 194.0 1.63948090893 -0.536765909643


In [59]:
centroid_psf("med_190218_091_095.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.37313611  656.96419936 ... 185.0 2.69056957316  -1.07461056645
  2 1565.61094604 1136.53699295 ... 193.0 1.65084636796 -0.544266646437


In [60]:
centroid_psf("med_190218_096_100.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.33345175 656.882605983 ... 188.0 2.66609993793  -1.06469106194
  2 1565.73654244 1136.90021495 ... 195.0 1.64519572862 -0.540543933228


In [61]:
centroid_psf("med_190218_101_105.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.33982103 656.886723916 ... 183.0 2.64757824253  -1.05712200863
  2 1565.62470804 1137.22342468 ... 192.0 1.63091205223 -0.531076355296


In [62]:
centroid_psf("med_190218_106_110.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1447.34835743   656.7425267 ... 185.0 2.64389842462  -1.05561191517
  2 1565.76978036 1136.49068614 ... 194.0 1.64773989445 -0.542221642006


In [63]:
centroid_psf("med_190218_111_115.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1  1461.8564016 658.909854484 ... 194.0 2.86127082862  -1.14139741785
  2 1566.19925113  1136.2549627 ... 195.0 1.62132093797 -0.524672478196


In [64]:
centroid_psf("med_190218_116_120.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux           mag      
--- ------------- ------------- ... ----- ------------- ---------------
  1 1475.19254673 660.867380114 ... 190.0 3.46410891917  -1.34897884682
  2 1566.09749614 1136.22051073 ... 193.0 1.62335078134 -0.526030935946


In [65]:
centroid_psf("med_190218_121_125.fits", mask = True)

 id   xcentroid     ycentroid   ...  peak      flux          mag     
--- ------------- ------------- ... ----- ------------- -------------
  1  1446.9324243 656.392151663 ... 185.0 2.57539472984 -1.0271095066
  2 1566.20851295 1136.16006459 ... 193.0 1.66643096656 -0.5544683183
